### Виконала: Хома Анна, КА-63
# Лабораторна робота №6
## Пошук власних чисел та векторів симетричних матриць
### 0. Умова

$ A = \begin{pmatrix}
6.37 & 1.12 & 0.95 & 1.225 \\
1.12 & 4.05 & 1.3 & 0.16 \\
0.95 & 1.30 & 5.55 & 2.1 \\
1.225 & 0.16 & 2.1 & 5.99
\end{pmatrix}$

Метод: **LR**

In [90]:
import numpy as np
from scipy.linalg import lu

In [91]:
A = np.matrix([[6.370, 1.120, 0.950, 1.225],
               [1.120, 4.050, 1.300, 0.160],
               [0.950, 1.300, 5.550, 2.100],
               [1.225, 0.160, 2.100, 5.990]])
EPS = 10**(-5)

### 1. Розв'язати часткову проблему власних значень: знайти найбільше та найменше власні числа та відповідні їм власні вектори степеневим методом або методом скалярних добутків.

In [92]:
def get_max_eig(A):
    x = np.random.rand(A.shape[1], 1)
    n = 1
    while True:
        ksi = np.dot(A, x)
        v = np.sign(np.float(ksi[0][0]/x[0][0])) * np.linalg.norm(ksi) / np.linalg.norm(x)
        w = ksi / np.linalg.norm(ksi)
        disc = np.dot(A, w) - np.dot(v, w)
        if np.linalg.norm(disc) < EPS:
            break
        x = w
        n += 1
    return v, w, disc, n
        
            
v, w, discrepancy, n = get_max_eig(A)

In [93]:
print('Максимальне власне число\n %s\n' % v)
print('Відповідний власний вектор\n %s\n' % w)
print('Вектор нев\'язки\n %s\n' % discrepancy)
print('Кількість ітерацій\n %s' % n)

Максимальне власне число
 9.25016822230202

Відповідний власний вектор
 [[0.53398979]
 [0.27222965]
 [0.55829952]
 [0.57362668]]

Вектор нев'язки
 [[-5.99758731e-06]
 [-1.78412622e-06]
 [ 2.74098954e-06]
 [ 3.76215015e-06]]

Кількість ітерацій
 24


In [94]:
def get_min_eig(A, v_max):
    B = A - v_max * np.eye(A.shape[0])
    v1, w1, disc, n = get_max_eig(B)
    return v1 + v_max, w1, disc, n

v1, w1, discrepancy1, n1 = get_min_eig(A, v)

In [95]:
print('Мінімальне власне число\n %s\n' % v1)
print('Відповідний власний вектор\n %s\n' % w1)
print('Вектор нев\'язки\n %s\n' % discrepancy)
print('Кількість ітерацій\n %s' % n)

Мінімальне власне число
 2.7606785229997675

Відповідний власний вектор
 [[-0.2058554 ]
 [ 0.69206825]
 [-0.55917114]
 [ 0.40742205]]

Вектор нев'язки
 [[-5.99758731e-06]
 [-1.78412622e-06]
 [ 2.74098954e-06]
 [ 3.76215015e-06]]

Кількість ітерацій
 24


### 2. Розвязати повну проблему методом LR розкладу.

In [96]:
iteration_counts = dict()

In [97]:
def lr_method(A):
    X = np.copy(A)
    v_prev = np.random.rand(X.shape[0])
    W = np.eye(X.shape[0])
    n = 1
    while True:
        p, l, u = lu(X)
        Y = np.dot(u, l)
        v = np.diag(Y)
        disc = v - v_prev
        print('Ітерація %s' % n)
        print('Похибка - %s' % np.linalg.norm(disc))
        print('Норма Фробініуса - %s' % np.linalg.norm(Y))
        print('Матриця L\n%s' % l)
        print('Матриця R\n%s\n' % u)
        
        if np.linalg.norm(disc) < EPS:
            iteration_counts['LR метод для власних чисел'] = n
            return v
        n += 1
        v_prev = v
        X = Y
            
v2 = lr_method(A)

Ітерація 1
Похибка - 10.22227937947569
Норма Фробініуса - 11.991750756447733
Матриця L
[[ 1.          0.          0.          0.        ]
 [ 0.17582418  1.          0.          0.        ]
 [ 0.14913658  0.29404215  1.          0.        ]
 [ 0.19230769 -0.01437413  0.38099004  1.        ]]
Матриця R
[[ 6.37        1.12        0.95        1.225     ]
 [ 0.          3.85307692  1.13296703 -0.05538462]
 [ 0.          0.          5.07518019  1.9335931 ]
 [ 0.          0.          0.          5.01694726]]

Ітерація 2
Похибка - 0.982864317164151
Норма Фробініуса - 12.082369771208162
Матриця L
[[ 1.          0.          0.          0.        ]
 [ 0.12035691  1.          0.          0.        ]
 [ 0.16254473  0.30838566  1.          0.        ]
 [ 0.13893614 -0.06568158  0.3357229   1.        ]]
Матриця R
[[ 6.94417975  1.38173174  1.4167128   1.225     ]
 [ 0.          4.02071213  0.94135487 -0.20282183]
 [ 0.          0.          5.29128035  1.79702315]
 [ 0.          0.          0.        

In [98]:
def reverse_iter(A, v):
    W = np.array([]).reshape((A.shape[0], 0))
    for value in v:
        n = 1
        x = np.random.rand(A.shape[0], 1)
        print('Власне число %s\n' % value)
        while True:
            y = np.linalg.solve(A - value * np.eye(A.shape[0]), x)
            y = y / np.linalg.norm(y)
            disc = y - x if np.linalg.norm(x - y) < np.linalg.norm(x + y) else x + y
            print('Ітерація %s' % n)
            print('Похибка - %s' % np.linalg.norm(disc))
            print('Наближений власний вектор\n%s' % y)
            if np.linalg.norm(disc) < EPS:
                W = np.append(W, y, axis=1)
                iteration_counts['Пошук власного вектора для власного числа %.2f' % value] = n
                print()
                break
            x = y
            n += 1
    return W
    
w2 = reverse_iter(A, v2)

Власне число 9.250168222338571

Ітерація 1
Похибка - 0.20961050552648075
Наближений власний вектор
[[0.53398819]
 [0.27222918]
 [0.55830025]
 [0.57362768]]
Ітерація 2
Похибка - 7.468261590855957e-16
Наближений власний вектор
[[0.53398819]
 [0.27222918]
 [0.55830025]
 [0.57362768]]

Власне число 5.498219992086382

Ітерація 1
Похибка - 1.559777287405595
Наближений власний вектор
[[ 0.77082779]
 [ 0.22490347]
 [-0.3545092 ]
 [-0.47913065]]
Ітерація 2
Похибка - 7.346046210536828e-05
Наближений власний вектор
[[ 0.77078768]
 [ 0.22488781]
 [-0.35455504]
 [-0.4791686 ]]
Ітерація 3
Похибка - 5.83564593073669e-10
Наближений власний вектор
[[ 0.77078768]
 [ 0.22488781]
 [-0.35455504]
 [-0.4791686 ]]

Власне число 4.4509332626459575

Ітерація 1
Похибка - 1.497688520697712
Наближений власний вектор
[[-0.27991013]
 [ 0.62957972]
 [ 0.49993068]
 [-0.52473709]]
Ітерація 2
Похибка - 3.515194548090805e-05
Наближений власний вектор
[[ 0.27994126]
 [-0.62957347]
 [-0.49991631]
 [ 0.52474166]]
Ітерація 3

In [99]:
discrepancy2 = np.dot(A, w2) - np.dot(np.diag(v2), w2.T).T

In [100]:
print('Власні числа\n %s\n' % v2)
print('Власні вектори\n %s\n' % w2)
print('Вектори нев\'язки\n %s\n' % discrepancy2)
print('Кількість ітерацій для:')
for key, value in iteration_counts.items():
    print('- %s - %s' % (key, value))

Власні числа
 [9.25016822 5.49821999 4.45093326 2.76067852]

Власні вектори
 [[ 0.53398819  0.77078768 -0.27994126 -0.20585396]
 [ 0.27222918  0.22488781  0.62957347  0.69206502]
 [ 0.55830025 -0.35455504  0.49991631 -0.55917371]
 [ 0.57362768 -0.4791686  -0.52474166  0.40742475]]

Вектори нев'язки
 [[ 1.33226763e-14  2.28007590e-05  8.28097470e-06  3.11972670e-14]
 [ 7.54951657e-15  6.65243203e-06 -1.86234852e-05 -1.04360964e-13]
 [ 1.42108547e-14 -1.04881335e-05 -1.47880818e-05  8.48210391e-14]
 [ 1.50990331e-14 -1.41743415e-05  1.55224434e-05 -6.19504448e-14]]

Кількість ітерацій для:
- LR метод для власних чисел - 64
- Пошук власного вектора для власного числа 9.25 - 2
- Пошук власного вектора для власного числа 5.50 - 3
- Пошук власного вектора для власного числа 4.45 - 3
- Пошук власного вектора для власного числа 2.76 - 2


### Перевірка результатів за допомогою сторонніх програмних пакетів

In [101]:
v, w = np.linalg.eig(A)
print('Власні числа\n %s\n' % v)
print('Власні вектори\n %s' % w)

Власні числа
 [9.25016822 2.76067852 5.49824957 4.45090368]

Власні вектори
 [[ 0.53398819  0.20585396 -0.77078768  0.27994126]
 [ 0.27222918 -0.69206502 -0.22488781 -0.62957347]
 [ 0.55830025  0.55917371  0.35455504 -0.49991631]
 [ 0.57362768 -0.40742475  0.4791686   0.52474166]]


### Висновок
В даній лабораторній роботі був реалізований LR метод пошуку власних значень та власних векторів. Були отримані результати з будь-якою наданою точністю, що свідчить про гарну реалізацію методу. Також було застосовано степеневий метод, який підтвердив правильність LR методу